# Manipulate x,y,z files

Transform them into something more manageable!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

from scipy.spatial import cKDTree
from scipy.interpolate import RegularGridInterpolator

In [ ]:
directory = '/mnt/home_geo/mguerri/Documents/itherc/fullea_et_al_2014_0/OUT_xyz/'

# import files
xyz_conductivity = np.loadtxt(directory+'Thermal_cond.xyz')
xyz_Trhopvels = np.loadtxt(directory+'Trhopvels.xyz', usecols=(0,1,2,3,4,5,6,7,8))

In Trhopvels

1. col: longitude
2. col: latitude
3. col: depth (in m, negative downwards)
4. col: temperature (C)
5. col: density (kg/m3)
6. col: pressure (MPa)
7. col: Vp (km/s)
8. col: Vs (km/s)

In [ ]:
# X,Y,Z nodes are identical for different files

Xcoords = np.unique(xyz_Trhopvels[:,0])
Ycoords = np.unique(xyz_Trhopvels[:,1])
Zcoords = np.unique(xyz_Trhopvels[:,2])

nx, ny, nz = Xcoords.size, Ycoords.size, Zcoords.size

# These grids are flipped in the y-direction for some reason
k_grid   = xyz_conductivity[:,-1].reshape(nx, ny, nz)[:,:,::-1]
T_grid   = xyz_Trhopvels[:,3].reshape(nx,ny,nz)[:,:,::-1]
rho_grid = xyz_Trhopvels[:,4].reshape(nx,ny,nz)[:,:,::-1]
P_grid   = xyz_Trhopvels[:,5].reshape(nx,ny,nz)[:,:,::-1]
Vp_grid  = xyz_Trhopvels[:,6].reshape(nx,ny,nz)[:,:,::-1]
Vs_grid  = xyz_Trhopvels[:,7].reshape(nx,ny,nz)[:,:,::-1]

print("dimensions: {}".format((nx,ny,nz)))
print("min/max:\n x {}\n y {}\n z {}".format((Xcoords.min(), Xcoords.max()),
                                             (Ycoords.min(), Ycoords.max()),
                                             (Zcoords.min(), Zcoords.max())))

Create interpolators

In [ ]:
k_interp   = RegularGridInterpolator((Xcoords, Ycoords, Zcoords), k_grid, method='linear')
T_interp   = RegularGridInterpolator((Xcoords, Ycoords, Zcoords), T_grid, method='linear')
rho_interp = RegularGridInterpolator((Xcoords, Ycoords, Zcoords), rho_grid, method='linear')
P_interp   = RegularGridInterpolator((Xcoords, Ycoords, Zcoords), P_grid, method='linear')
Vp_interp  = RegularGridInterpolator((Xcoords, Ycoords, Zcoords), Vp_grid, method='linear')
Vs_interp  = RegularGridInterpolator((Xcoords, Ycoords, Zcoords), k_grid, method='linear')

Now we interpolate onto a new grid

In [ ]:
import conduction

minX, maxX = Xcoords.min() + 1e-5, Xcoords.max() - 1e-5
minY, maxY = Ycoords.min() + 1e-5, Ycoords.max() - 1e-5
minZ, maxZ = Zcoords.min() + 1e-5, Zcoords.max() - 1e-5

mesh = conduction.Conduction3D((minX, minY, minZ), (maxX, maxY, maxZ), (nx, ny, nz))

print("bounding box: {}".format(mesh.dm.getBoundingBox()))

In [ ]:
# Get local mesh information

coords = mesh.coords

nx = mesh.nx
ny = mesh.ny
nz = mesh.nz

## Save to file

Save the fields to an HDF5 file to visualise in Paraview.

In [ ]:
d = {'conductivity': k_interp,
     'temperature': T_interp,
     'density': rho_interp,
     'pressure': P_interp,
     'P velocity': Vp_interp,
     'S velocity': Vs_interp}

filename = 'ireland.h5'
mesh.save_mesh_to_hdf5(filename)

for key in d:
    tempdict = dict()

    interp = d[key]
    field = interp(mesh.coords)
    
    tempdict[key] = field
    
    mesh.save_field_to_hdf5(filename, **tempdict)
    print("saved '{}' to {}".format(key, filename))
    
conduction.tools.generateXdmf(filename)

## Refinement (optional)

We refine in the $z$ direction.

In [ ]:
coords_old = coords.copy()

In [ ]:
def z_fun(z, C, k):
    return C*(1.0 - np.exp(-k*z))

zrange = np.linspace(-8.6, 1, nz)

Zcoords = z_fun(zrange, 5500., 0.5)
print Zcoords.min()

plt.plot(zrange, Zcoords)

In [ ]:
def zfun(z, C=5500.0, k=0.5):
    return C*(1.0 - np.exp(-k*z))


mesh = conduction.Conduction3D((minX, minY, zrange.min()), (maxX, maxY, zrange.max()), (nx, ny, nz))
mesh.refine(z_fn=zfun)

coords = mesh.coords

## Solve

In [ ]:
# material properties

k_local = k_interp(coords)
H_local = np.zeros_like(k_local)

k_local[k_local == 0] = 3.

mesh.update_properties(k_local, H_local)

In [ ]:
# BCs
Tmax = T_grid.max()
Tmin = 298.0

mesh.boundary_condition('maxZ', Tmin, flux=False)
mesh.boundary_condition('minZ', Tmax, flux=False)

In [ ]:
sol = mesh.solve(solver='cgs')

In [ ]:
H5_file = 'rectilinear_solution.h5'

mesh.save_mesh_to_hdf5(H5_file)
mesh.save_field_to_hdf5(H5_file, T=sol, k=mesh.diffusivity, H=mesh.heat_sources)
mesh.save_field_to_hdf5(H5_file, x=coords[:,0], y=coords[:,1], z=coords[:,2])
conduction.generateXdmf(H5_file)

In [ ]:
# maxZ - minZ
# z1 = np.linspace(Zcoords.min(), Zcoords.max(), nz)
# ratio = z1/Zcoords

coords[:,2].reshape(mesh.nx, mesh.ny, mesh.)
ratio = coords_old[:,2]/coords[:,2]
ratio

In [ ]:
mesh.save_field_to_hdf5(H5_file, zscale=ratio)
conduction.generateXdmf(H5_file)

In [ ]:
# mesh.save_mesh_to_hdf5('geometry.h5')
mesh.save_field_to_hdf5(H5_file, x=coords[:,0], y=coords[:,1], z=coords[:,2])

## Visualisation

This takes some resources for doing 3D particles in matplotlib, so we opt for some slices of the domain instead!

In [ ]:
fig = plt.figure(figsize=(10,8))
ax1 = fig.add_subplot(111, projection='3d', xlim=(minX, maxX), ylim=(minY, maxY),)
sca = ax1.scatter3D(coords[:,0], coords[:,1], coords[:,2], c=sol)
fig.colorbar(sca)

In [ ]:
prop = k_interp(coords)


fig = plt.figure(figsize=(10,8))
ax1 = fig.add_subplot(111, projection='3d', xlim=(minX, maxX), ylim=(minY, maxY),)
sca = ax1.scatter3D(coords[:,0], coords[:,1], coords[:,2], c=prop)
fig.colorbar(sca)